<img src="https://raw.githubusercontent.com/imedinam50/MCDI/main/MCDI.jpg">

<div style="text-align: right"> <H1> Proyecto final MCDI </H1></div>
<br><br><br>
<div style="text-align: right"> <H2> Asesor. Dr. Elio Atenógenes Villaseñor García </H2></div>
<br><br>

<H2>Ingeniería de datos</H2>
<br><br>
<H4><I> 
    Ismael Medina Muñoz
</I></H4>
<br>
<div style="text-align: right"> Fecha de entrega: 2022-0*-** </div>

# Introducción
## TODO

# Planteamiento del problema
La creación artística está siendo impactada por la inteligencia artificial. La música no es un campo ajeno a dicho impacto. La creación musical es un proceso que toma tiempo para el compositor. En un contexto de creación de obras artísticas usadas en productos como comerciales de productos, videojuegos, contenido audiovisual en internet, etc. se requiere de la producción acelerada.
Esta actividad de creación musical ha sido atribuida meramente a los humanos, tal cómo lo establece Morán Martinez, M (2009).

    La creación, la ejecución y la apreciación de la música obedecen fundamentalmente a la capacidad humana para descubrir patrones de sonido e identificarlos en ocasiones posteriores. Sin los procesos biológicos de percepción auditiva y sin consenso cultural sobre lo percibido, entre por lo menos algunos oyentes, no pueden existir ni música ni comunicación musical.

Si bien existen ya muchas investigaciones sobre la composición musical automatizada, la creación asistida e interactiva para los compositores es aún un reto para la investigación, tal como menciona Briot, J. (2021).

Para componer música necesitamos entonces de un generador de nuevas notas singulares o acordes basado en secuencias existentes, coherentes, pero principalmente consonantes, que ya sean parte de un corpus de entrenamiento.

En este proyecto se entrenará una red neuronal profunda para que sea un generador de un grupo de nuevas $n$-notas a un mismo tiempo a partir de dos vectores que representen secuencias de $n$-notas que ocurren a un mismo tiempo y cuya aparición en secuencia representaría grupos de $n$-notas que son consonantes. Este generador será un precursor a la asistencia en la composición de piezas de jazz para piano. Se escogió el género del jazz dado que éste representa un reto para los músicos por su complejidad y riqueza lingüística.

# Metodología propuesta y fundamentación
La composición musical consiste de crear secuencias de $n$-notas con $n = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}$ donde el silencio se representa por un conjunto de $0$-notas. Las $n$-notas ejecutadas en una secuencia $S$ deben ser agradables el oído humano, es decir, consonantes. Las partituras permiten registrar la música a través de escribir las $n$-notas colocadas en relación a su posición relativa en una secuencia $S$ dentro de los límites de un compás. Cada conjunto de $n$-notas tienen una duración que se mantienen bajo las reglas y límites propios de los compases.

La música es precursor del lenguaje hablado y por tanto mantiene una similitud con ésta, tal como establece Lozano(2013). Una oración sólo tiene coherencia si las palabras que la forman resultan en un significado para el receptor de la comunicación. Las oraciones forman párrafos y los signos de puntuación permiten agregar pausas y entonaciones que ayudan a que el cerebro del receptor consuma la información que el emisor pretendía.

Se escogió una fuente digital de partituras para modelar la secuencia $S$ de $n$-notas. El corpus provino entonces de __[musescore.com](https://musescore.com/)__. Desde este sitio se descargaron las partituras en formato **MSCZ** pertenecientes al género del jazz y de ahí se propone hacer la extracción de las $n$-notas asociadas al piano, ya que dichas partituras pueden tener más de un instrumento descrito en sus pentagramas.

Mediante la librería `ms3` se abrirán las partituras y de ellas se obtendrán las secuencias de $n$-notas asociadas al piano. Dichas $n$-notas ocurriendo en la misma posición de tiempo del compás se conocerán como rebanadas. Las secuencias de rebanadas de $n$-notas serán usadas para entrenar la red neuronal que predecirá la siguiente rebanada de $n$-notas en la secuencia. Este generador será el antecedente para la composición musical.

Este trabajo es similar al que ya se aplica para predecir la siguiente palabra o el siguiente número dada una secuencia previa y que ha sido modelado usando redes neuronales recurrentes (RNN).

Todos los conjuntos de datos intermedios en el procesamiento serán almacenados en archivos CSV dado el tiempo requerido para iterar el corpus y la capacidad que ofrece de segmentar el trabajo permitiendo retomar desde el paso último ejecutado.

# Implementación de la metodología

## Entrenamiento de la red neuronal
Con los datos ya codificados entonces fue posible hacer el entrenamiento de la red neuronal, lo primero es definir la creación de los tensores de entrada. 

### Los tensores de entrada
La definición de los tensores de entrada se realizó tomando como guía el trabajo de Ijas(2019). Este trabajo muestra la predicción de la siguiente palabra dada una secuencia de palabras previas. Es así que se generó una cantidad de $m$ vectores que representan el *One-Hot encoding* para las secuencias de $n$-notas. En este caso, los $m$ vectores son $3$.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import heapq
import random
import datetime
import os
from fractions import Fraction as frac

In [3]:
model = keras.models.load_model('c:/users/ismedina/OneDrive - Microsoft/documents/maestria/ProyectoFinal/modelos/FinalModel')

## Resultados adaptados a la ejecución musical con la variabilidad de modelos probabilísticos
### Extracción de los modelos de probabilidad

In [4]:
unique_n_notes_sequences = pd.read_csv('.\\data\\unique_n_notes_sequences.csv', index_col = 0)

In [5]:
unique_n_notes_sequences.head()

,prev_n_notes_id,actual_n_notes_id,next_n_notes_id
0,0,1,0
1,0,2,3
2,2,3,4
3,3,4,3
4,4,3,4


In [6]:
master_n_notes = pd.read_csv('.\\data\\master_n_notes.csv', index_col = 0)

In [7]:
master_n_notes.shape

(18229, 10)

In [8]:
master_n_notes.head()

,staff01_note01,staff01_note02,staff01_note03,staff01_note04,staff01_note05,staff02_note01,staff02_note02,staff02_note03,staff02_note04,staff02_note05
0,0,0,0,0,0,0,0,0,0,0
1,71,0,0,0,0,0,0,0,0,0
2,71,0,0,0,0,36,48,0,0,0
3,0,0,0,0,0,31,43,0,0,0
4,0,0,0,0,0,36,48,0,0,0


Así entonces definimos $\mathbb{x}$ tiene la forma de todos los `unique_n_notes_sequences` $ \times $ `1 conjuntos de n-notas previas` $ \times $ `cantidad de id's distintos` mientras que $\mathbb{y}$ es un vector donde el `id` del acorde más probable se registra.

In [9]:
## Esta es la definición de un tensor x de una dimensión que almacena el id de cada n-notas
x = np.zeros((unique_n_notes_sequences.shape[0], unique_n_notes_sequences.shape[1] - 2, master_n_notes.shape[0]), dtype=bool)

## Esta es la definición de un tensor x de una dimensión que almacena los ids de las n-notas previa y actual a
##    fin de definir secuencias con 2 grupos de n-notas
#x = np.zeros((unique_n_notes_sequences.shape[0], master_n_notes.shape[0]), dtype=bool)

## Esta es la definición de un tensor y de una dimensión que almacena el id de cada n-nota subsecuente a las x's n-notas previas
y = np.zeros((unique_n_notes_sequences.shape[0], master_n_notes.shape[0]), dtype=bool)

## Aquí se arma el One-Hot encoding para representar los acordes de entrada y de salida
for key, row in unique_n_notes_sequences.iterrows():
    ## Aquí se establece el acorde previo
    #x[key, 0, row[0]] = 1
    ## Aquí se establece el acorde actual
    #x[key, 1, row[1]] = 1
    ## Aquí se establece el acorde siguiente
    #y[key, row[2]] = 1
    
    ## Aquí se establece el acorde previo
    x[key, 0, row[0]] = 1
    ## Aquí se establece el acorde siguiente
    y[key, row[1]] = 1

### Partitura en 4/4

In [10]:
staff01_4_4_1_notes_durations = pd.read_csv('.\\data\\staff01_4_4_1_notes_durations.csv', index_col = 0)
staff01_4_4_2_notes_durations = pd.read_csv('.\\data\\staff01_4_4_1_notes_durations.csv', index_col = 0)
staff01_4_4_3_notes_durations = pd.read_csv('.\\data\\staff01_4_4_1_notes_durations.csv', index_col = 0)
staff01_4_4_4_notes_durations = pd.read_csv('.\\data\\staff01_4_4_1_notes_durations.csv', index_col = 0)
staff01_4_4_5_notes_durations = pd.read_csv('.\\data\\staff01_4_4_1_notes_durations.csv', index_col = 0)

staff02_4_4_1_notes_durations = pd.read_csv('.\\data\\staff02_4_4_1_notes_durations.csv', index_col = 0)
staff02_4_4_2_notes_durations = pd.read_csv('.\\data\\staff02_4_4_1_notes_durations.csv', index_col = 0)
staff02_4_4_3_notes_durations = pd.read_csv('.\\data\\staff02_4_4_1_notes_durations.csv', index_col = 0)
staff02_4_4_4_notes_durations = pd.read_csv('.\\data\\staff02_4_4_1_notes_durations.csv', index_col = 0)
staff02_4_4_5_notes_durations = pd.read_csv('.\\data\\staff02_4_4_1_notes_durations.csv', index_col = 0)

### Partitura en 2/4

In [11]:
ts_2_4 = '2/4'
staff01_2_4_1_notes_durations = pd.read_csv('.\\data\\staff01_2_4_1_notes_durations.csv', index_col = 0)
staff01_2_4_2_notes_durations = pd.read_csv('.\\data\\staff01_2_4_1_notes_durations.csv', index_col = 0)
staff01_2_4_3_notes_durations = pd.read_csv('.\\data\\staff01_2_4_1_notes_durations.csv', index_col = 0)
staff01_2_4_4_notes_durations = pd.read_csv('.\\data\\staff01_2_4_1_notes_durations.csv', index_col = 0)
staff01_2_4_5_notes_durations = pd.read_csv('.\\data\\staff01_2_4_1_notes_durations.csv', index_col = 0)

staff02_2_4_1_notes_durations = pd.read_csv('.\\data\\staff02_2_4_1_notes_durations.csv', index_col = 0)
staff02_2_4_2_notes_durations = pd.read_csv('.\\data\\staff02_2_4_1_notes_durations.csv', index_col = 0)
staff02_2_4_3_notes_durations = pd.read_csv('.\\data\\staff02_2_4_1_notes_durations.csv', index_col = 0)
staff02_2_4_4_notes_durations = pd.read_csv('.\\data\\staff02_2_4_1_notes_durations.csv', index_col = 0)
staff02_2_4_5_notes_durations = pd.read_csv('.\\data\\staff02_2_4_1_notes_durations.csv', index_col = 0)

### Partitura en 2/2

In [12]:
ts_2_2 = '2/2'
staff01_2_2_1_notes_durations = pd.read_csv('.\\data\\staff01_2_2_1_notes_durations.csv', index_col = 0)
staff01_2_2_2_notes_durations = pd.read_csv('.\\data\\staff01_2_2_1_notes_durations.csv', index_col = 0)
staff01_2_2_3_notes_durations = pd.read_csv('.\\data\\staff01_2_2_1_notes_durations.csv', index_col = 0)
staff01_2_2_4_notes_durations = pd.read_csv('.\\data\\staff01_2_2_1_notes_durations.csv', index_col = 0)
staff01_2_2_5_notes_durations = pd.read_csv('.\\data\\staff01_2_2_1_notes_durations.csv', index_col = 0)

staff02_2_2_1_notes_durations = pd.read_csv('.\\data\\staff02_2_2_1_notes_durations.csv', index_col = 0)
staff02_2_2_2_notes_durations = pd.read_csv('.\\data\\staff02_2_2_1_notes_durations.csv', index_col = 0)
staff02_2_2_3_notes_durations = pd.read_csv('.\\data\\staff02_2_2_1_notes_durations.csv', index_col = 0)
staff02_2_2_4_notes_durations = pd.read_csv('.\\data\\staff02_2_2_1_notes_durations.csv', index_col = 0)
staff02_2_2_5_notes_durations = pd.read_csv('.\\data\\staff02_2_2_1_notes_durations.csv', index_col = 0)

### Partitura en 3/4

In [13]:
ts_3_4 = '3/4'
staff01_3_4_1_notes_durations = pd.read_csv('.\\data\\staff01_3_4_1_notes_durations.csv', index_col = 0)
staff01_3_4_2_notes_durations = pd.read_csv('.\\data\\staff01_3_4_1_notes_durations.csv', index_col = 0)
staff01_3_4_3_notes_durations = pd.read_csv('.\\data\\staff01_3_4_1_notes_durations.csv', index_col = 0)
staff01_3_4_4_notes_durations = pd.read_csv('.\\data\\staff01_3_4_1_notes_durations.csv', index_col = 0)
staff01_3_4_5_notes_durations = pd.read_csv('.\\data\\staff01_3_4_1_notes_durations.csv', index_col = 0)

staff02_3_4_1_notes_durations = pd.read_csv('.\\data\\staff02_3_4_1_notes_durations.csv', index_col = 0)
staff02_3_4_2_notes_durations = pd.read_csv('.\\data\\staff02_3_4_1_notes_durations.csv', index_col = 0)
staff02_3_4_3_notes_durations = pd.read_csv('.\\data\\staff02_3_4_1_notes_durations.csv', index_col = 0)
staff02_3_4_4_notes_durations = pd.read_csv('.\\data\\staff02_3_4_1_notes_durations.csv', index_col = 0)
staff02_3_4_5_notes_durations = pd.read_csv('.\\data\\staff02_3_4_1_notes_durations.csv', index_col = 0)

### Partitura en 5/4

In [14]:
ts_5_4 = '5/4'
staff01_5_4_1_notes_durations = pd.read_csv('.\\data\\staff01_5_4_1_notes_durations.csv', index_col = 0)
staff01_5_4_2_notes_durations = pd.read_csv('.\\data\\staff01_5_4_1_notes_durations.csv', index_col = 0)
staff01_5_4_3_notes_durations = pd.read_csv('.\\data\\staff01_5_4_1_notes_durations.csv', index_col = 0)
staff01_5_4_4_notes_durations = pd.read_csv('.\\data\\staff01_5_4_1_notes_durations.csv', index_col = 0)
staff01_5_4_5_notes_durations = pd.read_csv('.\\data\\staff01_5_4_1_notes_durations.csv', index_col = 0)

staff02_5_4_1_notes_durations = pd.read_csv('.\\data\\staff02_5_4_1_notes_durations.csv', index_col = 0)
staff02_5_4_2_notes_durations = pd.read_csv('.\\data\\staff02_5_4_1_notes_durations.csv', index_col = 0)
staff02_5_4_3_notes_durations = pd.read_csv('.\\data\\staff02_5_4_1_notes_durations.csv', index_col = 0)
staff02_5_4_4_notes_durations = pd.read_csv('.\\data\\staff02_5_4_1_notes_durations.csv', index_col = 0)
staff02_5_4_5_notes_durations = pd.read_csv('.\\data\\staff02_5_4_1_notes_durations.csv', index_col = 0)

In [15]:
def define_durations(n_notes, staff = 1, time_signature = '4/4'):
    ## n_notes, lets define n
    n = np.count_nonzero(n_notes)
    n5_durations = [0, 0, 0, 0, 0]
    random_durations = [0, 0, 0, 0, 0]
    
    if time_signature == '4/4':
        staff01_1_notes_durations = staff01_4_4_1_notes_durations
        staff01_2_notes_durations = staff01_4_4_2_notes_durations
        staff01_3_notes_durations = staff01_4_4_3_notes_durations
        staff01_4_notes_durations = staff01_4_4_4_notes_durations
        staff01_5_notes_durations = staff01_4_4_5_notes_durations

        staff02_1_notes_durations = staff02_4_4_1_notes_durations
        staff02_2_notes_durations = staff02_4_4_2_notes_durations
        staff02_3_notes_durations = staff02_4_4_3_notes_durations
        staff02_4_notes_durations = staff02_4_4_4_notes_durations
        staff02_5_notes_durations = staff02_4_4_5_notes_durations
    if time_signature == '2/4':
        staff01_1_notes_durations = staff01_2_4_1_notes_durations
        staff01_2_notes_durations = staff01_2_4_2_notes_durations
        staff01_3_notes_durations = staff01_2_4_3_notes_durations
        staff01_4_notes_durations = staff01_2_4_4_notes_durations
        staff01_5_notes_durations = staff01_2_4_5_notes_durations

        staff02_1_notes_durations = staff02_2_4_1_notes_durations
        staff02_2_notes_durations = staff02_2_4_2_notes_durations
        staff02_3_notes_durations = staff02_2_4_3_notes_durations
        staff02_4_notes_durations = staff02_2_4_4_notes_durations
        staff02_5_notes_durations = staff02_2_4_5_notes_durations
    if time_signature == '2/2':
        staff01_1_notes_durations = staff01_2_2_1_notes_durations
        staff01_2_notes_durations = staff01_2_2_2_notes_durations
        staff01_3_notes_durations = staff01_2_2_3_notes_durations
        staff01_4_notes_durations = staff01_2_2_4_notes_durations
        staff01_5_notes_durations = staff01_2_2_5_notes_durations

        staff02_1_notes_durations = staff02_2_2_1_notes_durations
        staff02_2_notes_durations = staff02_2_2_2_notes_durations
        staff02_3_notes_durations = staff02_2_2_3_notes_durations
        staff02_4_notes_durations = staff02_2_2_4_notes_durations
        staff02_5_notes_durations = staff02_2_2_5_notes_durations
    if time_signature == '3/4':
        staff01_1_notes_durations = staff01_3_4_1_notes_durations
        staff01_2_notes_durations = staff01_3_4_2_notes_durations
        staff01_3_notes_durations = staff01_3_4_3_notes_durations
        staff01_4_notes_durations = staff01_3_4_4_notes_durations
        staff01_5_notes_durations = staff01_3_4_5_notes_durations

        staff02_1_notes_durations = staff02_3_4_1_notes_durations
        staff02_2_notes_durations = staff02_3_4_2_notes_durations
        staff02_3_notes_durations = staff02_3_4_3_notes_durations
        staff02_4_notes_durations = staff02_3_4_4_notes_durations
        staff02_5_notes_durations = staff02_3_4_5_notes_durations
    if time_signature == '5/4':
        staff01_1_notes_durations = staff01_5_4_1_notes_durations
        staff01_2_notes_durations = staff01_5_4_2_notes_durations
        staff01_3_notes_durations = staff01_5_4_3_notes_durations
        staff01_4_notes_durations = staff01_5_4_4_notes_durations
        staff01_5_notes_durations = staff01_5_4_5_notes_durations

        staff02_1_notes_durations = staff02_5_4_1_notes_durations
        staff02_2_notes_durations = staff02_5_4_2_notes_durations
        staff02_3_notes_durations = staff02_5_4_3_notes_durations
        staff02_4_notes_durations = staff02_5_4_4_notes_durations
        staff02_5_notes_durations = staff02_5_4_5_notes_durations
    
    if staff == 1:
        if n == np.int32(1):
            random_durations = random.choices(staff01_1_notes_durations.duration, 
                                              staff01_1_notes_durations.distribution, k=n)
        if n == np.int32(2):
            random_durations = random.choices(staff01_2_notes_durations.duration, 
                                              staff01_2_notes_durations.distribution, k=n)
        if n == np.int32(3):
            random_durations = random.choices(staff01_3_notes_durations.duration, 
                                              staff01_3_notes_durations.distribution, k=n)
        if n == np.int32(4):
            random_durations = random.choices(staff01_4_notes_durations.duration, 
                                              staff01_4_notes_durations.distribution, k=n)
        if n == np.int32(5):
            random_durations = random.choices(staff01_5_notes_durations.duration, 
                                              staff01_5_notes_durations.distribution, k=n)
    if staff == 2:
        if n == np.int32(1):
            random_durations = random.choices(staff02_1_notes_durations.duration, 
                                              staff02_1_notes_durations.distribution, k=n)
        if n == np.int32(2):
            random_durations = random.choices(staff02_2_notes_durations.duration, 
                                              staff02_2_notes_durations.distribution, k=n)
        if n == np.int32(3):
            random_durations = random.choices(staff02_3_notes_durations.duration, 
                                              staff02_3_notes_durations.distribution, k=n)
        if n == np.int32(4):
            random_durations = random.choices(staff02_4_notes_durations.duration, 
                                              staff02_4_notes_durations.distribution, k=n)
        if n == np.int32(5):
            random_durations = random.choices(staff02_5_notes_durations.duration, 
                                              staff02_5_notes_durations.distribution, k=n)
    for i in range(n):
        n5_durations[i] = random_durations[i]
    return n5_durations

In [16]:

def get_n_notes_id(n_notes):
    id = master_n_notes.loc[(master_n_notes.staff01_note01 == n_notes[0]) &
                         (master_n_notes.staff01_note02 == n_notes[1]) &
                         (master_n_notes.staff01_note03 == n_notes[2]) &
                         (master_n_notes.staff01_note04 == n_notes[3]) &
                         (master_n_notes.staff01_note05 == n_notes[4]) &
                         (master_n_notes.staff02_note01 == n_notes[5]) &
                         (master_n_notes.staff02_note02 == n_notes[6]) &
                         (master_n_notes.staff02_note03 == n_notes[7]) &
                         (master_n_notes.staff02_note04 == n_notes[8]) &
                         (master_n_notes.staff02_note05 == n_notes[9])].index[0]
    return id

In [17]:
top_n = 5

n_notes_4_4_sequence = pd.DataFrame(columns=['staff01_notes', 'staff01_durations', 
                                           'staff02_notes','staff02_durations'], dtype=object)

n_notes_2_4_sequence = pd.DataFrame(columns=['staff01_notes', 'staff01_durations', 
                                           'staff02_notes','staff02_durations'], dtype=object)

n_notes_2_2_sequence = pd.DataFrame(columns=['staff01_notes', 'staff01_durations', 
                                           'staff02_notes','staff02_durations'], dtype=object)

n_notes_3_4_sequence = pd.DataFrame(columns=['staff01_notes', 'staff01_durations', 
                                           'staff02_notes','staff02_durations'], dtype=object)

n_notes_5_4_sequence = pd.DataFrame(columns=['staff01_notes', 'staff01_durations', 
                                           'staff02_notes','staff02_durations'], dtype=object)

main_n_notes_id_sequence = []

## Aquí obtenemos un valor aleatorio de una tripleta de secuencias de n-notas:
##    - Previo
##    - Actual
##    - Siguiente
## NOTA: Dado que x está definido como una secuencia de un elemento de n-notas,
##    sólo se extrae el valor "Previo" no se requiere de una selección de algún elemento de la tripleta
##encoded_n_notes = x[np.random.choice(range(len(x)), size=1)[0]]

## Aquí traemos el tensor del acorde inicial (codificado)
encoded_n_notes = x[np.random.choice(range(len(x)), size=1)[0]].copy()
## Aquí traemos el id de la secuencia de n-notes inicial
n_notes_index = encoded_n_notes.argmax()
## En esta variable vamos a comparar la repetición de secuencias de n-notas que se repitan (2, 3, 4, 5)
main_n_notes_id_sequence.append(n_notes_index)
## Aquí traemos la secuencia de n-notes inicial desde el id
all_n_notes = master_n_notes.loc[n_notes_index]
## Aquí separamos la secuencia completa de n-notes en las 2 manos, representados por cada pentagrama
staff01_n_notes = all_n_notes[0:5].values
staff02_n_notes = all_n_notes[5:10].values
## Aquí definimos la duración de las notas en cada secuencia de n-notes por pentagrama
staff01_4_4_durations = define_durations(staff01_n_notes, 1)
staff02_4_4_durations = define_durations(staff02_n_notes, 2)
staff01_2_4_durations = define_durations(staff01_n_notes, 1, time_signature = '2/4')
staff02_2_4_durations = define_durations(staff02_n_notes, 2, time_signature = '2/4')
staff01_2_2_durations = define_durations(staff01_n_notes, 1, time_signature = '2/2')
staff02_2_2_durations = define_durations(staff02_n_notes, 2, time_signature = '2/2')
staff01_3_4_durations = define_durations(staff01_n_notes, 1, time_signature = '3/4')
staff02_3_4_durations = define_durations(staff02_n_notes, 2, time_signature = '3/4')
staff01_5_4_durations = define_durations(staff01_n_notes, 1, time_signature = '5/4')
staff02_5_4_durations = define_durations(staff02_n_notes, 2, time_signature = '5/4')

n_notes_4_4_sequence.loc[len(n_notes_4_4_sequence.index)] = (staff01_n_notes, staff01_4_4_durations,
                                                        staff02_n_notes, staff02_4_4_durations)
n_notes_2_4_sequence.loc[len(n_notes_2_4_sequence.index)] = (staff01_n_notes, staff01_2_4_durations, 
                                                        staff02_n_notes, staff02_2_4_durations)
n_notes_2_2_sequence.loc[len(n_notes_2_2_sequence.index)] = (staff01_n_notes, staff01_2_2_durations, 
                                                        staff02_n_notes, staff02_2_2_durations)
n_notes_3_4_sequence.loc[len(n_notes_3_4_sequence.index)] = (staff01_n_notes, staff01_3_4_durations, 
                                                        staff02_n_notes, staff02_3_4_durations)
n_notes_5_4_sequence.loc[len(n_notes_5_4_sequence.index)] = (staff01_n_notes, staff01_5_4_durations, 
                                                        staff02_n_notes, staff02_5_4_durations)

for i in range(63):
    ## Aquí seleccionamos sólo la secuencia de n-notas "Previo" ([-2:])
    new_prediction = model.predict(np.expand_dims(encoded_n_notes[-2:], axis=0))
    ## Aquí aplanamos el arreglo a una sola dimensión
    new_prediction = np.squeeze(new_prediction, axis=0)
    ## Aquí obtenemos las predicciones en formato de logaritmo
    log_prediction = np.log(new_prediction)
    ## Aquí obtenemos las predicciones en formato exponencial
    exp_prediction = np.exp(log_prediction)        
    ## Aquí obtenemos la relación de predicciones más probables
    prediction = exp_prediction / np.sum(exp_prediction)

    ## Aquí obtenemos los Id's las "n" predicciones más probables de las "m" que la red neuronal tiene
    ##    como tensor de salida
    n_notes_indexes = heapq.nlargest(top_n, range(len(prediction)), prediction.take)
    n_notes_index = n_notes_indexes[0]
    
    ## Aquí evitamos los ciclos de notas repetidas de inmediato
    if n_notes_index == staff01_n_notes.tolist() + staff02_n_notes.tolist():
        n_notes_index = n_notes_indexes[1]       
    ## Aquí evitamos los ciclos de 5 secuencias de n-notas repetidas
    if len(main_n_notes_id_sequence) > 8:
        if main_n_notes_id_sequence[len(main_n_notes_id_sequence)-9:-4] == main_n_notes_id_sequence[-4:] + [n_notes_index]:
            ##print(main_n_notes_id_sequence[len(main_n_notes_id_sequence)-9:-4], main_n_notes_id_sequence[-4:] + [n_notes_index])
            n_notes_index = np.random.choice(n_notes_indexes, 1, p = [0, 0.25, 0.25, 0.25, 0.25])
            ##print(main_n_notes_id_sequence[len(main_n_notes_id_sequence)-9:-4], main_n_notes_id_sequence[-4:] + [n_notes_index])
    ## Aquí evitamos los ciclos de 4 secuencias de n-notas repetidas
    if len(main_n_notes_id_sequence) > 6:
        if main_n_notes_id_sequence[len(main_n_notes_id_sequence)-7:-3] == main_n_notes_id_sequence[-3:] + [n_notes_index]:
            ##print(main_n_notes_id_sequence[len(main_n_notes_id_sequence)-7:-3], main_n_notes_id_sequence[-3:] + [n_notes_index])
            n_notes_index = np.random.choice(n_notes_indexes, 1, p = [0, 0.25, 0.25, 0.25, 0.25])
            ##print(main_n_notes_id_sequence[len(main_n_notes_id_sequence)-7:-3], main_n_notes_id_sequence[-3:] + [n_notes_index])
    if len(main_n_notes_id_sequence) > 4:
        if main_n_notes_id_sequence[len(main_n_notes_id_sequence)-5:-2] == main_n_notes_id_sequence[-2:] + [n_notes_index]:
            ##print(main_n_notes_id_sequence[len(main_n_notes_id_sequence)-5:-2], main_n_notes_id_sequence[-2:] + [n_notes_index])
            n_notes_index = np.random.choice(n_notes_indexes, 1, p = [0, 0.25, 0.25, 0.25, 0.25])
            ##print(main_n_notes_id_sequence[len(main_n_notes_id_sequence)-5:-2], main_n_notes_id_sequence[-2:] + [n_notes_index])
    if len(main_n_notes_id_sequence) > 2:
        if main_n_notes_id_sequence[len(main_n_notes_id_sequence)-3:-1] == main_n_notes_id_sequence[-1:] + [n_notes_index]:
            ##print(main_n_notes_id_sequence[len(main_n_notes_id_sequence)-3:-1], main_n_notes_id_sequence[-1:] + [n_notes_index])
            n_notes_index = np.random.choice(n_notes_indexes, 1, p = [0, 0.25, 0.25, 0.25, 0.25])
            ##print(main_n_notes_id_sequence[len(main_n_notes_id_sequence)-3:-1], main_n_notes_id_sequence[-1:] + [n_notes_index])
        
    main_n_notes_id_sequence.append(int(n_notes_index))
    
    ## Compara con la nota anterior, si es igual entonces no la agrega
    encoded_n_notes = np.zeros((1, master_n_notes.shape[0]), dtype=bool)
    encoded_n_notes[0][n_notes_index] = True

    ## Aquí traemos la secuencia de n-notes inicial desde el id
    all_n_notes = master_n_notes.loc[int(n_notes_index)]
    ## Aquí separamos la secuencia completa de n-notes en las 2 manos, representados por cada pentagrama
    staff01_n_notes = all_n_notes[0:5].values
    staff02_n_notes = all_n_notes[5:10].values
    ## Aquí definimos la duración de las notas en cada secuencia de n-notes por pentagrama
    staff01_4_4_durations = define_durations(staff01_n_notes, 1)
    staff02_4_4_durations = define_durations(staff02_n_notes, 2)
    staff01_2_4_durations = define_durations(staff01_n_notes, 1, time_signature = '2/4')
    staff02_2_4_durations = define_durations(staff02_n_notes, 2, time_signature = '2/4')
    staff01_2_2_durations = define_durations(staff01_n_notes, 1, time_signature = '2/2')
    staff02_2_2_durations = define_durations(staff02_n_notes, 2, time_signature = '2/2')
    staff01_3_4_durations = define_durations(staff01_n_notes, 1, time_signature = '3/4')
    staff02_3_4_durations = define_durations(staff02_n_notes, 2, time_signature = '3/4')
    staff01_5_4_durations = define_durations(staff01_n_notes, 1, time_signature = '5/4')
    staff02_5_4_durations = define_durations(staff02_n_notes, 2, time_signature = '5/4')

    n_notes_4_4_sequence.loc[len(n_notes_4_4_sequence.index)] = (staff01_n_notes, staff01_4_4_durations,
                                                            staff02_n_notes, staff02_4_4_durations)
    n_notes_2_4_sequence.loc[len(n_notes_2_4_sequence.index)] = (staff01_n_notes, staff01_2_4_durations, 
                                                            staff02_n_notes, staff02_2_4_durations)
    n_notes_2_2_sequence.loc[len(n_notes_2_2_sequence.index)] = (staff01_n_notes, staff01_2_2_durations, 
                                                            staff02_n_notes, staff02_2_2_durations)
    n_notes_3_4_sequence.loc[len(n_notes_3_4_sequence.index)] = (staff01_n_notes, staff01_3_4_durations, 
                                                            staff02_n_notes, staff02_3_4_durations)
    n_notes_5_4_sequence.loc[len(n_notes_5_4_sequence.index)] = (staff01_n_notes, staff01_5_4_durations, 
                                                            staff02_n_notes, staff02_5_4_durations)

1/1 [==============================] - 0s 134ms/step


In [18]:
time_stamp = datetime.datetime.now().strftime("%Y%m%d_%H_%M_%S_%f")
folder = '.\\output\\' + time_stamp
if not os.path.exists(folder):
    os.makedirs(folder)

n_notes_4_4_sequence.to_csv(folder + '\\n_notes_4_4_sequence.csv')
n_notes_2_4_sequence.to_csv(folder + '\\n_notes_2_4_sequence.csv')
n_notes_2_2_sequence.to_csv(folder + '\\n_notes_2_2_sequence.csv')
n_notes_3_4_sequence.to_csv(folder + '\\n_notes_3_4_sequence.csv')
n_notes_5_4_sequence.to_csv(folder + '\\n_notes_5_4_sequence.csv')

# Referencias

* Briot, J. (2021). From artificial neural networks to deep learning for music generation: history, concepts and trends. *Neural Comput & Applic 33*, 36-65.
* Ijas, A. H. (2 de Noviembre de 2019). *Build a simple predictive keyboard using python and Keras*. Obtenido de medium.com: https://medium.com/analytics-vidhya/build-a-simple-predictive-keyboard-using-python-and-keras-b78d3c88cffb
* Lozano Cruz, O. S.-G. (6 de Junio de 2013). El cerebro y la música. *Rev Med UV, Enero - Junio 2013*, págs. 17-22.
* Morán Martínez, M. C. (1 de Noviembre de 2009). Psicología y Música: inteligencia musical y desarrollo estético*. *Revista Digital Universitaria*, págs. 1-13.

